In [1]:
from bs4 import BeautifulSoup
import re
import requests

import pandas as pd
import numpy as np

import spacy
import spacy.cli

In [2]:
from time import sleep
from random import randint

In [3]:
my_array = np.array([5, 5, 5, 4, 4])
l = 4
(my_array-l).max() > 1

False

In [4]:
dic_sentimentos = {}

lista_sent = [('alegria','alegria', 0), ('medo','medo', 0), ('nojo','nojo', 0), ('raiva','raiva', 0), ('tristeza','tristeza', 0)]
#lista_sent = [('raiva','raiva', 0), ('nojo','nojo', 0), ('medo','medo', 0)]

In [5]:
#aux = 'maca'
#while (lista_sent != [] and aux != 'medo'):
#    print('lala')

In [6]:
def verifica_balanco():
    if len(dic_sentimentos) < 5:
        return 1
    
    qt_dicionario = pd.DataFrame(pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts()).sort_index()['sent'].tolist()
    qt_dicionario = np.array(qt_dicionario)
    
    menor_valor = qt_dicionario.min()
    
    if (qt_dicionario-menor_valor).min() > 1:
        return 0
    else:
        return 1    

In [7]:
def balancear_sentimentos():
    #print('balancear chegou')
    #print(pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts())
    aux = pd.DataFrame(pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts()).sort_index()['sent'].tolist()
    sent_desbalanceado = ['alegria','medo','nojo','raiva','tristeza'][0]
    m_desbalanceado = aux[0]

    for i in range(1,len(aux)):
        v = aux[i]
        
        if v < m_desbalanceado:
            m_desbalanceado = v
            sent_desbalanceado = ['alegria','medo','nojo','raiva','tristeza'][i]
            
    #if (aux-m_desbalanceado).max() > 1:
    #    k = 0
    #    print('aqui')
    #else:
    #    k = 1
                
    #print('balancear saida')
    #print(sent_desbalanceado)
    return sent_desbalanceado

In [8]:
def busca_elemento_lista(sentimento):
    #print('busca chegou')
    global lista_sent
    
    sent_origem = 'nulo'
    
    i = 0
    
    while (i < len(lista_sent) and sent_origem != sentimento):
        s, sent_origem, h = lista_sent[i]
        #print('sent origem ' + sent_origem)
        i += 1    

    #print('i :' + str(i))
    return i-1    

In [9]:
def cria_dicionario():
    global lista_sent
    global d
    
    s, o, h = lista_sent.pop(0)
    #print('\nANTES')
    #print('s ' + s)
    #print('o ' + o)
    
    if s != o:
        lista_sent.insert(0, (s, o, h))
        sd = balancear_sentimentos()
        #print('sd '+ sd)           
        indice = busca_elemento_lista(sd)
        #print('indice '+ str(indice))
        s, o, h = lista_sent.pop(indice)

    #print('\nDEPOIS')
    #print('s ' + s)
    #print('o ' + o)

    #verifica se a palavra está no dicionario
    if s in dic_sentimentos:
        return
    
    if h > d:
        dic_sentimentos[s] =  o
        return
    
    #caso as verificações estejam ok, adiciona no dicionário e faz a busca 
    
    dic_sentimentos[s] =  o
    #print('\nDICIONARIO ')
    #print(dic_sentimentos)
    
    url = "https://www.sinonimos.com.br/" + s
    soup = BeautifulSoup(requests.get(url).text, "html5lib")
    retorno = soup.find_all('a', class_ ='sinonimo')
    lista_sent = lista_sent + list(pd.Series(retorno).apply(lambda x: (str(x).split('>')[1][:-3], o, h+1)))
    
    #sleep(randint(1,3))
    
    #cria_dicionario() 

In [10]:
d = 1
r = 1

while lista_sent and verifica_balanco():    
    cria_dicionario()
    #print(len(lista_sent))

/home/karine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [11]:
(pd.DataFrame.from_dict(dic_sentimentos, orient='index')).to_csv('dic_sentimento_1_niveis_atualizado.txt', sep='\t' )

In [12]:
pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts()

alegria     296
nojo        268
tristeza    225
raiva       182
medo        181
Name: sent, dtype: int64

In [13]:
pd.DataFrame(pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts()).sort_index()['sent'].tolist()

[296, 181, 268, 182, 225]